In [ ]:
# notebooks/main_workflow.ipynb (converted to .py for versioning, run in Jupyter)
from agents.rag_agent import answer_with_rag
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email

# === USER QUERY ===
query = "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. What might be the issue?"

# === STEP 1: Use RAG Agent ===
rag_output = answer_with_rag(query)
print("\n--- RAG Agent Output ---\n")
print(rag_output)

# === STEP 2: Use Web Agent ===
web_output = search_web(rag_output)
print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 3: Use Summarizer Agent ===
combined = f"RAG Output:\n{rag_output}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)
print("\n--- Summary for Email ---\n")
print(summary)

# === STEP 4: Use Email Agent ===
send_email("Child Mental Health Assessment Summary", summary)


In [1]:
import sys
import os

# Go one level up to include the project root
sys.path.append(os.path.abspath(".."))


In [ ]:
!pip install langchain langgraph faiss-cpu duckduckgo-search unstructured pypdf


In [ ]:
!pip install ollama


In [ ]:
pip install -U langchain-community

In [ ]:
pip install pymupdf


In [ ]:
pip install sentence-transformers

In [ ]:
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.rag_agent import build_rag_agent

In [ ]:
!pip install -U langchain-huggingface

In [ ]:
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import search_doctor
from config import SENDER_NAME
import time

sender_name = SENDER_NAME
# === Build RAG Once ===
rag_chain = build_rag_agent()  # ✅ Only builds once


# === USER QUERY ===
query = "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. What might be the issue?"
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")
# === STEP 1: Use RAG Agent ===
rag_output = rag_chain.invoke(query)
print("\n--- RAG Agent Output ---\n")
print(rag_output)

# === STEP 2: Use Web Agent ===
time.sleep(10)  # prevent DuckDuckGo rate-limiting
web_output = search_web(rag_output["result"])  # ✅ correct

print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 3: Use Summarizer Agent ===
combined = f"RAG Output:\n{rag_output}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)
#print("\n--- Summary for Email ---\n")
# print(summary)

# === STEP 4: Collect Email Info ===
# recipient_name = input("Enter recipient's name: ")
# recipient_email = input("Enter recipient's email: ")

# Personalize summary
personalized_summary = summary.replace("[Recipient]", recipient_name).replace("[Your Name]", sender_name)

# === STEP 5: Follow-up RAG for Doctor Type ===
followup_query = f"What type of doctor should be consulted for this: {rag_output['result']}"
doctor_type = rag_chain.invoke(followup_query)["result"].strip()

#doctor_type = answer_with_rag(f"What type of doctor should be consulted for this: {rag_output}").strip()

# === STEP 6: Ask for Location and Search Doctor ===
# location = input("Enter city or pin code to search for doctors nearby: ")
doctor_info = search_doctor(doctor_type, location)

# === STEP 7: Final Email Body ===
final_email_body = f"{personalized_summary}\n\n--- Doctor Recommendation ---\n{doctor_info}"

# === STEP 8: Send Final Email ===
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email_body, recipient_email)
print("\n--- Summary for Email ---\n")
print(summary)

In [ ]:
print(rag_output)

In [ ]:
ollama list


In [ ]:
pip install duckduckgo-search


In [ ]:
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import search_doctor
from config import SENDER_NAME
import time

sender_name = SENDER_NAME
# === Build RAG Once ===
rag_chain = build_rag_agent()  # ✅ Only builds once


# === USER QUERY ===
query = "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. What might be the issue?"
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")
# === STEP 1: Use RAG Agent ===
rag_output = rag_chain.invoke(query)
print("\n--- RAG Agent Output ---\n")
print(rag_output)

# === STEP 2: Use Web Agent ===
  # prevent DuckDuckGo rate-limiting
web_output = search_web(rag_output["result"])  # ✅ correct

print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 3: Use Summarizer Agent ===
combined = f"RAG Output:\n{rag_output}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# === STEP 4: Collect Email Info ===

# Personalize summary
personalized_summary = f"Dear {recipient_name},\n\n" + summary.replace("[Your Name]", sender_name)

# === STEP 5: Use RAG to get only doctor specialties (clean & structured) ===
followup_query = (
    "From the earlier analysis, list only the doctor specialties or professional roles required to treat this child. "
    "Return them as a comma-separated list only. Do not include explanations."
)
doctor_titles = rag_chain.invoke(followup_query)["result"].strip()

# Optional retry if the output is a sentence
if "." in doctor_titles or len(doctor_titles.split()) > 15:
    followup_query_retry = (
        "List only the doctor types (comma-separated, no sentence) that should treat a child with these symptoms."
    )
    doctor_titles = rag_chain.invoke(followup_query_retry)["result"].strip()

print("\n--- Doctor Specialties Identified ---\n")
print(doctor_titles)

# === STEP 6: Use doctor_search_agent with the identified titles ===
doctor_info_links = search_doctor(doctor_titles, location)

# === STEP 7: Final Email Body ===
final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {doctor_titles}

{doctor_info_links}
"""

# === STEP 8: Send Final Email ===
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email_body, recipient_email)
print("\n--- Summary for Email ---\n")
print(summary)

In [ ]:
##new
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import search_doctor
from config import SENDER_NAME
import time

sender_name = SENDER_NAME

# === Build RAG Once ===
rag_chain = build_rag_agent()  # ✅ Only builds once

# === USER INPUT ===
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")

# === STEP 1: Use RAG Agent (Single Call for Diagnosis + Doctors) ===
combined_query = (
    "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention.\n"
    "What might be the issue?\n"
    "Also, based on your analysis, list the doctor specialties (comma-separated only, no explanation) "
    "who should be consulted for this case."
)
rag_response = rag_chain.invoke(combined_query)["result"].strip()

# === STEP 2: Parse Diagnosis and Doctor Titles ===
if "\n\n" in rag_response:
    diagnosis_part, doctor_titles = rag_response.split("\n\n", 1)
else:
    lines = rag_response.splitlines()
    diagnosis_part = "\n".join(lines[:-1])
    doctor_titles = lines[-1]

diagnosis = diagnosis_part.strip()
doctor_titles = doctor_titles.strip().rstrip(".")

print("\n--- Diagnosis ---\n")
print(diagnosis)
print("\n--- Doctor Specialties Identified ---\n")
print(doctor_titles)

# === STEP 3: Use Web Agent (Optional) ===
web_output = search_web(diagnosis)

print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 4: Summarize ===
combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# === STEP 5: Personalize Summary ===
personalized_summary = f"Dear {recipient_name},\n\n" + summary.replace("[Your Name]", sender_name)

# === STEP 6: Get Doctor Info ===
doctor_info_links = search_doctor(doctor_titles, location)

# === STEP 7: Final Email Body ===
final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {doctor_titles}

{doctor_info_links}
"""

# === STEP 8: Send Final Email ===
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email_body, recipient_email)
print("\n--- Summary for Email ---\n")
print(summary)


In [ ]:
pip install beautifulsoup4 requests


In [ ]:
import importlib
import agents.doctor_search_agent
importlib.reload(agents.doctor_search_agent)

from agents.doctor_search_agent import get_doctor_specialty, search_doctor


In [ ]:
## Proper run this firt in the morning
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import get_doctor_specialty, search_doctor, normalize_roles

from config import SENDER_NAME

sender_name = SENDER_NAME

# === Build RAG Once ===
rag_chain = build_rag_agent()

# === USER INPUT ===
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")

# === STEP 1: Use RAG Agent for Diagnosis ===
query = (
    "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. "
    "What might be the issue?"
)
rag_response = rag_chain.invoke(query)["result"]
diagnosis = rag_response.strip()

print("\n--- Diagnosis ---\n")
print(diagnosis)
# === STEP 2: Get Doctor Specialties (RAG Follow-up) ===
specialties = get_doctor_specialty(diagnosis, rag_chain)
print("\n--- Doctor Specialties Identified ---\n")
print(specialties)
print("🛠️ DEBUG: Helper functions loaded")

# === STEP 3: Web Search (Optional) ===
web_output = search_web(diagnosis)
print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 4: Summarize ===
combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# === STEP 5: Personalize Summary ===
personalized_summary = f"Dear {recipient_name},\n\n" + summary.replace("[Your Name]", sender_name)

# === STEP 6: Normalize & Search for Doctors ===
normalized_specialties = normalize_roles(specialties)
print("\n--- Normalized Doctor Specialties for Practo ---\n")
print(normalized_specialties)

doctor_info_links = search_doctor(", ".join(normalized_specialties), location)



# === STEP 7: Final Email Body ===
final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {', '.join(specialties)}

{doctor_info_links}
"""

# === STEP 8: Send Email ===
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email_body, recipient_email)

# === Optional Preview ===
print("\n✅ Email Sent! --- Summary Preview Below ---\n")
print(summary)


In [ ]:
import agents.doctor_search_agent as dsa

print(dir(dsa))


In [ ]:
import importlib
import agents.doctor_search_agent as dsa
importlib.reload(dsa)
print(dir(dsa))  # Make sure you see 'get_doctor_specialty'


In [ ]:
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from config import SENDER_NAME

# --- Global cache for RAG ---
rag_chain = build_rag_agent()

# --- INPUT ---
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")

# --- STEP 1: Diagnosis via RAG ---
query = (
    "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. "
    "What might be the issue?"
)
rag_response = rag_chain.invoke(query)["result"]
diagnosis = rag_response.strip()
print("\n--- Diagnosis ---\n")
print(diagnosis)

# --- STEP 2: Extract Doctor Roles (from same chain) ---
followup_query = (
    "List only the doctor types (e.g., Pediatrician, Child Psychologist) to treat this child. "
    "Return as comma-separated values only with no explanation."
)
doctor_raw = rag_chain.invoke(followup_query)["result"].strip()

# Cleanup roles
if "." in doctor_raw or len(doctor_raw.split()) > 20:
    retry_query = "List doctor types like Pediatrician, Neurologist. No explanation. Comma-separated."
    doctor_raw = rag_chain.invoke(retry_query)["result"].strip()

specialties = [s.strip() for s in re.split(r",|and", doctor_raw) if s.strip()]
print("\n--- Doctor Specialties Identified ---\n")
print(specialties)

# --- STEP 3: Web Search ---
web_output = search_web(diagnosis)
print("\n--- Web Search Output ---\n")
print(web_output)

# --- STEP 4: Summarize ---
combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# --- STEP 5: Personalize ---
personalized_summary = f"Dear {recipient_name},\n\n" + summary.replace("[Your Name]", SENDER_NAME)

# --- STEP 6: Doctor Lookup ---
def search_doctor(doctor_types, location):
    roles = [r.strip() for r in doctor_types.split(",")]
    city = location.strip().replace(" ", "-").lower()
    headers = {"User-Agent": "Mozilla/5.0"}

    output = ""
    for role in roles[:2]:  # Limit to 2 roles for speed
        q = quote(role)
        url = f"https://www.practo.com/{city}/doctors?specialization={q}"
        try:
            r = requests.get(url, headers=headers, timeout=5)
            soup = BeautifulSoup(r.text, "html.parser")
            cards = soup.select("div.card")[:3]

            if not cards:
                output += f"\n❌ No {role} found in {location.title()}. [Search manually]({url})\n"
                continue

            output += f"\n🔍 Top {role}s in {location.title()}:\n"
            for card in cards:
                name = card.select_one("h2")
                spec = card.select_one(".u-color--gray-dark")
                clinic = card.select_one(".clinic-card .u-bold")

                output += f"- {name.text.strip() if name else 'N/A'} ({spec.text.strip() if spec else role}) – {clinic.text.strip() if clinic else 'Unknown Clinic'}\n"
        except Exception as e:
            output += f"\n❌ Error for {role}: {e}\n"

    return output

# --- STEP 7: Doctor Search ---
doctor_links = search_doctor(", ".join(specialties), location)

# --- STEP 8: Compose Email ---
final_email = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {', '.join(specialties)}

{doctor_links}
"""

# --- STEP 9: Send Email ---
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email, recipient_email)

# --- Preview ---
print("\n✅ Email Sent! --- Summary Preview Below ---\n")
print(summary)


In [ ]:
import time
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import get_doctor_specialty, search_doctor, normalize_roles
from config import SENDER_NAME
from agents.doctor_search_agent import search_multiple_doctors

def log_time(label, start_time):
    print(f"⏱️ {label} took {time.time() - start_time:.2f} seconds\n")

def main():
    sender_name = SENDER_NAME

    # === Build RAG Once ===
    start = time.time()
    rag_chain = build_rag_agent()
    log_time("Build RAG Agent", start)

    # === USER INPUT ===
    recipient_name = input("Enter recipient's name: ")
    recipient_email = input("Enter recipient's email: ")
    location = input("Enter city or pin code to search for doctors nearby: ")

    # === STEP 1: Use RAG Agent for Diagnosis ===
    start = time.time()
    query = (
        "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. "
        "What might be the issue?"
    )
    rag_response = rag_chain.invoke(query)["result"]
    diagnosis = rag_response.strip()
    log_time("Diagnosis RAG Query", start)

    print("\n--- Diagnosis ---\n")
    print(diagnosis)

    # === STEP 2: Get Doctor Specialties ===
    start = time.time()
    specialties = get_doctor_specialty(diagnosis, rag_chain)
    log_time("Get Doctor Specialties", start)

    print("\n--- Doctor Specialties Identified ---\n")
    print(specialties)

    # === STEP 3: Web Search (Optional) ===
    start = time.time()
    web_output = search_web(diagnosis)
    log_time("Web Search", start)

    print("\n--- Web Search Output ---\n")
    print(web_output)

    # === STEP 4: Summarize ===
    start = time.time()
    combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
    summary = summarize_text(combined)
    log_time("Summary Generation", start)

    # === STEP 5: Personalize Summary ===
    personalized_summary = (
        f"Dear {recipient_name},\n\n"
        + summary.replace("[Your Name]", sender_name).replace("[Recipient]", recipient_name)
    )

    # === STEP 6: Normalize & Search for Doctors ===
    start = time.time()
    normalized_specialties = normalize_roles(specialties)
    print("\n--- Normalized Doctor Specialties for Practo ---\n")
    print(normalized_specialties)

    doctor_info_links = search_multiple_doctors(specialties, location)
    log_time("Doctor Search", start)

    # === STEP 7: Final Email Body ===
    final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {', '.join(specialties)}

{doctor_info_links}
"""

    # === STEP 8: Send Email ===
    start = time.time()
    send_email(
        "Child Mental Health Assessment Summary & Doctor Recommendation",
        final_email_body,
        recipient_email,
    )
    log_time("Email Sent", start)

    # === Optional Preview ===
    print("\n✅ Email Sent! --- Summary Preview Below ---\n")
    print(summary)


if __name__ == "__main__":
    main()


In [2]:
import os
import json
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import get_doctor_specialty, search_doctor, normalize_roles
from config import SENDER_NAME


# === CACHE SETUP ===
CACHE_FILE = "cache_session.json"

def load_cache():
    return json.load(open(CACHE_FILE)) if os.path.exists(CACHE_FILE) else {}

def save_cache(data):
    with open(CACHE_FILE, "w") as f:
        json.dump(data, f, indent=2)

cache = load_cache()

# === Build RAG Once ===
print("RAG already initialized – using cached instance")
rag_chain = build_rag_agent()

# === USER INPUT ===
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")

query = (
    "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. "
    "What might be the issue?"
)

# === STEP 1: Diagnosis with cache ===
if "diagnosis" not in cache:
    print("💡 Running diagnosis...")
    rag_response = rag_chain.invoke(query)["result"]
    cache["diagnosis"] = rag_response.strip()
    save_cache(cache)
else:
    print("✅ Loaded cached diagnosis")

diagnosis = cache["diagnosis"]
print("\n--- Diagnosis ---\n")
print(diagnosis)

# === STEP 2: Doctor Specialties with cache ===
if "specialties" not in cache:
    print("💡 Extracting specialties...")
    raw_roles = get_doctor_specialty(diagnosis, rag_chain)
    cache["specialties"] = raw_roles
    save_cache(cache)
else:
    print("✅ Loaded cached specialties")

specialties = cache["specialties"]

# === STEP 3: Web Search (cached inside web_agent.py) ===
print("🌐 Searching Duckduckgo...")
web_output = search_web(diagnosis)
print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 4: Summarize diagnosis + web ===
combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# === STEP 5: Personalize Email Body ===
sender_name = SENDER_NAME
personalized_summary = f"Dear {recipient_name},\n\n" + summary.replace("[Your Name]", sender_name)

# === STEP 6: Normalize + Search Doctor Info ===
normalized_specialties = list(set(normalize_roles(specialties)))
print("\n--- Normalized Doctor Specialties for Practo ---\n")
print(normalized_specialties)
print("Extracted specialties:", specialties)
print("Normalized specialties:", normalized_specialties)

# Unique cache key for combination of specialties and location
cache_key = f"{','.join(normalized_specialties)}@{location}"

if cache_key not in cache:
    print("🔎 Searching Practo for each specialty...")
    results = []
    for spec in normalized_specialties:
        try:
            print(f"🔎 Searching: {spec} in {location}")
            search_result = search_doctor(location, spec)
            if "❌" not in search_result.lower():
                results.append(f"\n🩺 **{spec.title()}** →\n{search_result}")
            else:
                results.append(f"\n❌ **{spec.title()}** → Not found\n{search_result}")

        except Exception as e:
            results.append(f"\n❌ **{spec.title()}** → Error: {e}\n")
    doctor_info_links = "\n".join(results)
    cache[cache_key] = doctor_info_links
    save_cache(cache)
else:
    print("✅ Loaded cached doctor search results")
    doctor_info_links = cache[cache_key]

# === STEP 7: Final Email Body ===
final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {', '.join(specialties)}

{doctor_info_links}
"""

# === STEP 8: Send Email ===
send_email("Child Mental Health Assessment Summary & Doctor Recommendation", final_email_body, recipient_email)

# === Optional Preview ===
print("\n✅ Email Sent! --- Summary Preview Below ---\n")
print(summary)


PDF Path: D:\Surya\Myworkspace\Project\rag-mcp-agents\data\MentalHealth.pdf
Absolute Path: D:\Surya\Myworkspace\Project\rag-mcp-agents\data\MentalHealth.pdf
Exists: True


D:\Surya\Myworkspace\Project\rag-mcp-agents\agents\summarizer_agent.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=MODEL_NAME)


RAG already initialized – using cached instance
🔄 Loading embeddings...
✅ Loading existing FAISS vector store from disk...
✅ Ollama LLM loaded


Enter recipient's name:  Priya Patel
Enter recipient's email:  surajsuresh1509@gmail.com
Enter city or pin code to search for doctors nearby:  mumbai


✅ Loaded cached diagnosis

--- Diagnosis ---

The 12-year-old child's symptoms suggest Attention Deficit Hyperactivity Disorder (ADHD). The child may have difficulty paying attention to details, sustaining attention to tasks or activities, following through on instructions, organizing tasks and activities, and playing or engaging in leisure activities quietly. These are common symptoms of ADHD as indicated by the Vanderbilt ADHD Teacher Rating Scale. It is recommended to consult a healthcare professional for a proper diagnosis and treatment plan.
✅ Loaded cached specialties
🌐 Searching Duckduckgo...
🔍 Performing new search using DuckDuckGo

--- Web Search Output ---

No relevant results found.

--- Normalized Doctor Specialties for Practo ---

['Psychiatrist', 'neurologist', 'pediatrician', 'Psychologist']
Extracted specialties: ['pediatrician', 'neurologist', 'Psychiatrist', 'Psychologist']
Normalized specialties: ['Psychiatrist', 'neurologist', 'pediatrician', 'Psychologist']
🔎 Searc

C:\Users\user\AppData\Local\Temp\ipykernel_28832\2719092943.py:91: RuntimeWarning: coroutine 'search_doctor' was never awaited
  search_result = search_doctor(location, spec)


✅ Email sent successfully.

✅ Email Sent! --- Summary Preview Below ---

 Hi there,

I've been analyzing some health-related information regarding your child. The symptoms observed suggest that your child may have Attention Deficit Hyperactivity Disorder (ADHD). This condition often makes it hard for children to focus on tasks, follow instructions, and engage in quiet activities.

To confirm this diagnosis and discuss the best treatment options, I recommend consulting a healthcare professional as soon as possible. They can provide a thorough evaluation based on their expertise.

Please let me know if you have any questions or need further assistance.

Best regards,
[Your Name]


In [ ]:
pip install webdriver-manager

In [ ]:
from agents.doctor_search_agent import search_doctor

output = search_doctor("Child Psychologist, Pediatrician", "Mumbai")
print(output)


In [ ]:
from agents import doctor_search_agent
print(dir(doctor_search_agent))


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

url = "https://www.practo.com/mumbai/doctors?specialization=Psychologist"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# now parse doctor_cards like before
driver.quit()


In [ ]:
!pip install selenium

In [ ]:
pip install selenium beautifulsoup4


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

# Set Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)

# Set the path to chromedriver
service = Service("C:/WebDrivers/chromedriver.exe")  # use forward slashes or double backslashes

# Launch browser
driver = webdriver.Chrome(service=service, options=options)

# Open the URL
url = "https://www.practo.com/mumbai/doctors?specialization=Psychologist"
driver.get(url)

# Extract HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Do your parsing here
print(soup.title.text)  # Example: Print page title

# Close the browser
driver.quit()


In [ ]:
from agents.doctor_search_agent import search_doctor

print(search_doctor("Psychiatrist", "mumbai"))


In [ ]:
!pip install playwright

In [ ]:
import os
import json
from agents.rag_agent import build_rag_agent
from agents.web_agent import search_web
from agents.summarizer_agent import summarize_text
from agents.email_agent import send_email
from agents.doctor_search_agent import get_doctor_specialty, search_doctor, normalize_roles
from config import SENDER_NAME

# === CACHE SETUP ===
CACHE_FILE = "cache_session.json"

def load_cache():
    return json.load(open(CACHE_FILE)) if os.path.exists(CACHE_FILE) else {}

def save_cache(data):
    with open(CACHE_FILE, "w") as f:
        json.dump(data, f, indent=2)

cache = load_cache()

# === Build RAG Once ===
print("RAG already initialized – using cached instance")
rag_chain = build_rag_agent()

# === USER INPUT ===
recipient_name = input("Enter recipient's name: ")
recipient_email = input("Enter recipient's email: ")
location = input("Enter city or pin code to search for doctors nearby: ")

query = (
    "A 12-year-old child is making careless mistakes in schoolwork and struggles to maintain attention. "
    "What might be the issue?"
)

# === STEP 1: Diagnosis with cache ===
if "diagnosis" not in cache:
    print("💡 Running diagnosis...")
    rag_response = rag_chain.invoke(query)["result"]
    cache["diagnosis"] = rag_response.strip()
    save_cache(cache)
else:
    print("✅ Loaded cached diagnosis")

diagnosis = cache["diagnosis"]
print("\n--- Diagnosis ---\n")
print(diagnosis)

# === STEP 2: Doctor Specialties with cache ===
if "specialties" not in cache:
    print("💡 Extracting specialties...")
    raw_roles = get_doctor_specialty(diagnosis, rag_chain)
    cleaned_roles = normalize_roles(raw_roles)
    cache["specialties"] = cleaned_roles
    save_cache(cache)
else:
    print("✅ Loaded cached specialties")

specialties = cache["specialties"]
normalized_specialties = list(set(specialties))  # Deduplicated list

# === STEP 3: Web Search ===
print("🔎 Searching Google...")
web_output = search_web(diagnosis)
print("\n--- Web Search Output ---\n")
print(web_output)

# === STEP 4: Summarize diagnosis + web ===
combined = f"Diagnosis:\n{diagnosis}\n\nWeb Results:\n{web_output}"
summary = summarize_text(combined)

# === STEP 5: Personalize Email Body ===
sender_name = SENDER_NAME
personalized_summary = f"Dear {recipient_name},\n\n{summary.strip()}\n\nRegards,\n{sender_name}"

# === STEP 6: Doctor Search on Practo ===
print("\n--- Normalized Doctor Specialties for Practo ---\n")
print(normalized_specialties)

cache_key = f"{','.join(normalized_specialties)}@{location}"

if cache_key not in cache:
    print("🔎 Searching Practo...")
    try:
        doctor_info_links = search_doctor(", ".join(normalized_specialties), location)
    except Exception as e:
        doctor_info_links = f"❌ No verified doctors found. [Search manually](https://www.practo.com/search?results_for=doctor&query={','.join(normalized_specialties)}&city={location})"
        print("❌ Practo search failed:", e)
    cache[cache_key] = doctor_info_links
    save_cache(cache)
else:
    print("✅ Loaded cached doctor search results")
    doctor_info_links = cache[cache_key]

# === STEP 7: Final Email Body ===
final_email_body = f"""{personalized_summary}

--- Doctor Recommendation ---
Specialist(s) recommended: {', '.join(normalized_specialties)}

{doctor_info_links}
"""

# === STEP 8: Send Email ===
send_email(
    "Child Mental Health Assessment Summary & Doctor Recommendation",
    final_email_body,
    recipient_email
)

# === Optional Preview ===
print("\n✅ Email Sent! --- Summary Preview Below ---\n")
print(summary)


In [3]:
from agents.web_agent import search_web

result = search_web("pediatricians in Mumbai")
print("--- Web Search Output ---\n")
print(result)


🔍 Performing new search using DuckDuckGo
--- Web Search Output ---

Best Pediatricians Near Me in Mumbai - Practo - //duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.practo.com%2Fmumbai%2Fpediatrician&rut=d59bda92456911313f4071a6ac73089d6f1dff1bb46205fa761783b5e92ec284

Top 10 Pediatrician in Mumbai, Child Specialist in Mumbai, Top 10 ... - //duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.365doctor.in%2Fpediatrician%2Din%2Dmumbai&rut=f8c19c05056bc123b5bb72d086d8c3425eaa396350fad2f8258e3df1223de0fb

The Children's Hospital in Mumbai: Best Care for Your Child - //duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.thechildrenshospitalmumbai.com%2F&rut=339ee5be15d6cb1a2e5e686658d4218c9a61291e8fdc6430f6462986e592403d

Pediatrician in Mumbai - Best Child Specialist - Top List - Book ... - //duckduckgo.com/l/?uddg=https%3A%2F%2Fwww.lybrate.com%2Fmumbai%2Fpediatrician&rut=a34db981f023adba630f24c95f31ee0c81001c65c451b1bb43749a58a700d5ba

Top 10 Pediatricians in Mumbai - Best Child Specialist - //duckduckgo.com/l/?udd

In [4]:
!pip install transformers peft datasets accelerate bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2025.3.0 which is incompatible.



   ---------------------------------------- 0.0/72.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.2 MB 2.7 MB/s eta 0:00:27
    --------------------------------------- 1.6/72.2 MB 3.0 MB/s eta 0:00:24
   - -------------------------------------- 2.4/72.2 MB 3.2 MB/s eta 0:00:23
   - -------------------------------------- 3.4/72.2 MB 3.5 MB/s eta 0:00:20
   -- ------------------------------------- 4.2/72.2 MB 3.6 MB/s eta 0:00:19
   -- ------------------------------------- 5.0/72.2 MB 3.7 MB/s eta 0:00:19
   --- ------------------------------------ 6.0/72.2 MB 3.8 MB/s eta 0:00:18
   --- ------------------------------------ 7.1/72.2 MB 3.9 MB/s eta 0:00:17
   ---- ----------------------------------- 7.6/72.2 MB 3.9 MB/s eta 0:00:17
   ---- ----------------------------------- 8.1/72.2 MB 3.6 MB/s eta 0:00:18
   ----- ---------------------------------- 9.2/72.2 MB 3.7 MB/s eta 0:00:18
   ----- ---

In [3]:
#Load the dataset
from datasets import load_dataset, Dataset
import json

# Load your local file
with open("C:/Users/user/Downloads/mental_health_qa.json", "r") as f:
    data = json.load(f)

# Convert to HuggingFace dataset
dataset = Dataset.from_list(data)


In [4]:
#Load Model in 4-bit Quantized Form
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import prepare_model_for_kbit_training

model_name = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

model = prepare_model_for_kbit_training(model)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

C:\Users\user\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--tiiuae--falcon-rw-1b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/2e/3a/2e3a78ac75bfe23c7017cb7792215cef508b32914302fce14d30c8b3b95f46fd/3a0d68f0309c8f7ec913f51edf8bf2eca849477c9f53db727f49ffa2f6019251?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1753208154&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzIwODE1NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8yZS8zYS8yZTNhNzhhYzc1YmZlMjNjNzAxN2NiNzc5MjIxNWNlZjUwOGIzMjkxNDMwMmZjZTE0ZDMwYzhiM2I5NWY0NmZkLzNhMGQ2OGYwMzA5YzhmN2VjOTEzZjUxZWRmOGJmMmVjYTg0OTQ3N2M5ZjUzZGI3MjdmNDlmZmEyZjYwMTkyNTE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=UMJr0hNSoSGzQDgI%7EyK%7Ep3bZAGYGnMB6KNl6Zunvnetf7PXkbGm%7E8nxMJQ29OJ7ky54fdH2G0QOMG2SJhttkc6Sgzt8OTyEr7IPvM5HYT37t7J4JM%7EDUQF%7EuCWK4jo6LDq11CWO9zWNorFxi0wmcekhJx%7Ezn6gPYwjZlkJ6Jghdu3o

pytorch_model.bin:   2%|1         | 52.4M/2.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

RuntimeError: quant_type must be nf4 on CPU, got fp4

In [5]:
# Apply LoRA (Parameter Efficient Fine-tuning)
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


NameError: name 'model' is not defined

In [ ]:
#Prepare Data for Training
def generate_prompt(example):
    return f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""

def tokenize(example):
    return tokenizer(generate_prompt(example), truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize)


In [ ]:
# Fine-tune the Model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()
